In [1]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import sqlalchemy as db
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table, Float, Boolean, Text, MetaData
from sqlalchemy.orm import sessionmaker, relationship, declarative_base, join
from google.colab import drive


drive.mount('/content/drive')

db_file_path = '/content/drive/MyDrive/GTDb.db'

engine = create_engine(f'sqlite:///{db_file_path}')
Base = declarative_base()

Session = sessionmaker(bind=engine)
session = Session()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
class Region(Base):
    __tablename__ = 'region'

    id = Column(Integer, primary_key=True)
    region_txt = Column(String, nullable=False)

    countries = relationship("Country", back_populates="region")


class Country(Base):
    __tablename__ = 'country'

    id = Column(Integer, primary_key=True)
    country_txt = Column(String)
    region_id = Column(Integer, ForeignKey('region.id'))

    region = relationship("Region", back_populates="countries")
    cities = relationship("City", back_populates="country")


class City(Base):
    __tablename__ = 'city'

    id = Column(Integer, primary_key=True)
    city = Column(String)
    country_id = Column(Integer, ForeignKey('country.id'))

    country = relationship("Country", back_populates="cities")
    incidents = relationship("Incident", back_populates="city")


class TypeAttack(Base):
    __tablename__ = 'type_attack'

    id = Column(Integer, primary_key=True)
    attacktype1_txt = Column(String)

    incidents = relationship("Incident", back_populates="type_attack")


class Corp(Base):
    __tablename__ = 'corp'

    id = Column(Integer, primary_key=True)
    corp1 = Column(String)

    incidents = relationship("Incident", back_populates="corp")


class Natlty(Base):
    __tablename__ = 'natlty'

    id = Column(Integer, primary_key=True)
    natlty1_txt = Column(String)

    incidents = relationship("Incident", back_populates="natlty")


class WeaponType(Base):
    __tablename__ = 'weapon_type'

    id = Column(Integer, primary_key=True)
    weaptype1_txt = Column(String)

    weapon_sub_types = relationship("WeaponSubType", back_populates="weapon_type")


class WeaponSubType(Base):
    __tablename__ = 'weapon_sub_type'

    id = Column(Integer, primary_key=True)
    weapsubtype1_txt = Column(String)
    weapon_type_id = Column(Integer, ForeignKey('weapon_type.id'))

    weapon_type = relationship("WeaponType", back_populates="weapon_sub_types")
    incidents = relationship("Incident", back_populates="weapon_sub_type")


class Propextent(Base):
    __tablename__ = 'propextent'

    id = Column(Integer, primary_key=True)
    propextent_txt = Column(String)

    incidents = relationship("Incident", back_populates="propextent")


class Claimmode(Base):
    __tablename__ = 'claimmode'

    id = Column(Integer, primary_key=True)
    claimmode_txt = Column(String)

    incidents = relationship("Incident", back_populates="claimmode")


class Target(Base):
    __tablename__ = 'target'

    id = Column(Integer, primary_key=True)
    target1 = Column(Text)

    incidents = relationship("Incident", back_populates="target")


class TargetType(Base):
    __tablename__ = 'target_type'

    id = Column(Integer, primary_key=True)
    targtype1_txt = Column(String)

    target_sub_types = relationship("TargetSubType", back_populates="target_type")


class TargetSubType(Base):
    __tablename__ = 'target_sub_type'

    id = Column(Integer, primary_key=True)
    targsubtype1_txt = Column(String)
    target_type_id = Column(Integer, ForeignKey('target_type.id'))

    target_type = relationship("TargetType", back_populates="target_sub_types")
    incidents = relationship("Incident", back_populates="target_sub_type")


class TypeIncident(Base):
    __tablename__ = 'type_incident'

    id = Column(Integer, primary_key=True)
    alternative_txt = Column(String)

    incidents = relationship("Incident", back_populates="type_incident")


class Incident(Base):
    __tablename__ = 'incident'

    id = Column(Integer, primary_key=True)
    iyear = Column(Integer)
    imonth = Column(Integer)
    iday = Column(Integer)
    city_id = Column(Integer, ForeignKey('city.id'))
    type_incident_id = Column(Integer, ForeignKey('type_incident.id'))
    type_attack_id = Column(Integer, ForeignKey('type_attack.id'))
    target_sub_type_id = Column(Integer, ForeignKey('target_sub_type.id'))
    corp_id = Column(Integer, ForeignKey('corp.id'))
    target_id = Column(Integer, ForeignKey('target.id'))
    natlty_id = Column(Integer, ForeignKey('natlty.id'))
    claimmode_id = Column(Integer, ForeignKey('claimmode.id'))
    weapon_sub_type_id = Column(Integer, ForeignKey('weapon_sub_type.id'))
    propextent_id = Column(Integer, ForeignKey('propextent.id'))

    city = relationship("City", back_populates="incidents")
    type_incident = relationship("TypeIncident", back_populates="incidents")
    type_attack = relationship("TypeAttack", back_populates="incidents")
    target_sub_type = relationship("TargetSubType", back_populates="incidents")
    corp = relationship("Corp", back_populates="incidents")
    target = relationship("Target", back_populates="incidents")
    natlty = relationship("Natlty", back_populates="incidents")
    claimmode = relationship("Claimmode", back_populates="incidents")
    weapon_sub_type = relationship("WeaponSubType", back_populates="incidents")
    propextent = relationship("Propextent", back_populates="incidents")
    other_data = relationship("OtherData", back_populates="incident", uselist = False)
    international = relationship("InternationalData", back_populates="incident", uselist=False)
    place_spec = relationship("PlaceSpec", back_populates="incident", uselist=False)
    hostages = relationship("Hostages", back_populates="incident", uselist=False)
    attack_spec = relationship("AttackSpec", back_populates="incident", uselist=False)
    victims = relationship("Victims", back_populates="incident", uselist=False)
    properties = relationship("Properties", back_populates="incident", uselist=False)


class OtherData(Base):
    __tablename__ = 'other_data'

    id = Column(Integer, ForeignKey('incident.id'), primary_key=True)
    addnotes = Column(Text)
    scite1 = Column(Text)
    scite2 = Column(Text)
    scite3 = Column(Text)

    incident = relationship("Incident", back_populates="other_data", uselist=False)


class InternationalData(Base):
    __tablename__ = 'international_data'

    id = Column(Integer, ForeignKey('incident.id'), primary_key=True)
    INT_LOG = Column(Integer)
    INT_IDEO = Column(Integer)
    INT_MISC = Column(Integer)
    INT_ANY = Column(Integer)

    incident = relationship("Incident", back_populates="international", uselist=False)


class PlaceSpec(Base):
    __tablename__ = 'placespec'

    id = Column(Integer, ForeignKey('incident.id'), primary_key=True)
    latitude = Column(Float)
    longitude = Column(Float)
    specificity = Column(Integer)
    vicinity = Column(Integer)
    location = Column(Text)
    summary = Column(Text)
    motive = Column(Text)

    incident = relationship("Incident", back_populates="place_spec", uselist=False)


class Hostages(Base):
    __tablename__ = 'hostages'

    id = Column(Integer, ForeignKey('incident.id'), primary_key=True)
    ishostkid = Column(Integer)
    nhostkid = Column(Integer)
    nhostkidus = Column(Integer)
    ndays = Column(Integer)
    ransom = Column(Integer)
    ransomamt = Column(Integer)
    ransompaid = Column(Integer)
    hostkidoutcome_txt = Column(String)

    incident = relationship("Incident", back_populates="hostages", uselist=False)


class AttackSpec(Base):
    __tablename__ = 'attackspec'

    id = Column(Integer, ForeignKey('incident.id'), primary_key=True)
    crit1 = Column(Boolean)
    crit2 = Column(Boolean)
    crit3 = Column(Boolean)
    doubtterr = Column(Integer)
    multiple = Column(Boolean)
    success = Column(Boolean)
    suicide = Column(Boolean)

    incident = relationship("Incident", back_populates="attack_spec", uselist=False)


class Victims(Base):
    __tablename__ = 'victims'

    id = Column(Integer, ForeignKey('incident.id'), primary_key=True)
    nkill = Column(Integer)
    nkillus = Column(Integer)
    nkilter = Column(Integer)
    nwound = Column(Integer)
    nwoundus = Column(Integer)
    nwoundte = Column(Integer)

    incident = relationship("Incident", back_populates="victims", uselist=False)

class Properties(Base):
    __tablename__ = 'properties'

    id = Column(Integer, ForeignKey('incident.id'), primary_key=True)
    properties = Column(Integer)
    propvalue = Column(Integer)
    propcomment = Column(Text)

    incident = relationship("Incident", back_populates="properties", uselist=False)


In [3]:
with Session() as session:
    query = db.select(
        Victims.nkill, Incident.id, Incident.iyear, Incident.imonth, Incident.iday, Region.region_txt,
        Country.country_txt, City.city, Corp.corp1, PlaceSpec.summary
        ).outerjoin(City, City.id == Incident.city_id
        ).outerjoin(PlaceSpec, PlaceSpec.id == Incident.id
        ).outerjoin(Corp, Corp.id == Incident.corp_id
        ).outerjoin(Country, Country.id == City.country_id
        ).outerjoin(Region, Region.id == Country.region_id
        ).outerjoin(Victims, Victims.id == Incident.id
        ).where(Country.country_txt == 'Russia').where(Victims.nkill > 50)
    result = session.execute(query)
    for row in enumerate(result):
        print(row)

(0, (121, 59561, 1995, 6, 14, 'Eastern Europe', 'Russia', 'Budyonnovsk', None, None))
(1, (64, 68677, 1999, 3, 19, 'Eastern Europe', 'Russia', 'Vladikavkaz', 'Civilian', "03/19/1999: A bomb was detonated in the central market of Vladikavkaz, North Ossetia, Russia. The explosion killed 64 people and injured 104. The equ ... (249 characters truncated) ... he Jordanian Chechnya field commander 'Hattab' were thought to be the masterminds. Two men, aged 18 and 33 were sought in connection with the attack."))
(2, (64, 69394, 1999, 9, 4, 'Eastern Europe', 'Russia', 'Buynaksk', 'Russian Army', '09/04/1999: In Buynaksk, Dagestan, Russia, sixty-four people were killed and over 100 were wounded when a GAZ-52 vehicle containing one ton of explos ... (455 characters truncated) ... ere released under an amnesty law after receiving three-year sentences. In 2006, Magomed Salikhov, charged with organizing the attack, was acquitted.'))
(3, (90, 69411, 1999, 9, 9, 'Eastern Europe', 'Russia', 'Pechatniki'

In [4]:
query = db.select(Incident.iyear, Victims.nkill,).outerjoin(Victims, Victims.id == Incident.id)

In [5]:
year_victims = pd.read_sql(query, engine)

In [6]:
year_victims.head()

,iyear,nkill
0,1970,1.0
1,1970,0.0
2,1970,1.0
3,1970,NaN
4,1970,NaN


In [7]:
year_victims.fillna({'nkill': 0}, inplace=True)

In [8]:
yearly_data = year_victims.groupby('iyear', as_index=False)['nkill'].sum()

In [9]:
fig_line = px.line(yearly_data, x='iyear', y='nkill', title='Количество жертв по годам',
                   labels={'iyear': 'Год', 'nkill': 'Количество жертв'})
fig_line.show()

In [10]:
fig_histogram = px.histogram(yearly_data, x='iyear', y='nkill', title='Гистограмма количества жертв по годам',
                              labels={'iyear': 'Год', 'nkill': 'Количество жертв'},
                              histfunc='sum', nbins=len(yearly_data['iyear'].unique()))
fig_histogram.show()

In [11]:
query = db.select(InternationalData.INT_ANY, InternationalData.INT_IDEO,
                  InternationalData.INT_LOG, InternationalData.INT_MISC,
                  Victims.nkill, Properties.propvalue
                  ).outerjoin(Incident, Incident.id == InternationalData.id
                  ).outerjoin(Victims, Victims.id == Incident.id
                  ).outerjoin(Properties, Properties.id == Incident.id)

In [12]:
table = pd.read_sql(query, engine)

In [13]:
corr_matrix = table.corr()

fig = px.imshow(corr_matrix,
                title="Корреляционная матрица",
                labels=dict(color="Correlation"),
                width=500, height=500,
                x=corr_matrix.columns,
                y=corr_matrix.columns,
                color_continuous_scale='RdBu_r',
                zmin=-1, zmax=1)

fig.show()

In [14]:
x = np.arange(0, 5, 0.1)
def f(x):
    return x**2

fig = px.scatter(x=x, y=f(x))
fig.show()

In [15]:
go.Scatter(x=x, y=f(x))

Scatter({
    'x': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
                1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7,
                2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4. , 4.1,
                4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9]),
    'y': array([0.000e+00, 1.000e-02, 4.000e-02, 9.000e-02, 1.600e-01, 2.500e-01,
                3.600e-01, 4.900e-01, 6.400e-01, 8.100e-01, 1.000e+00, 1.210e+00,
                1.440e+00, 1.690e+00, 1.960e+00, 2.250e+00, 2.560e+00, 2.890e+00,
                3.240e+00, 3.610e+00, 4.000e+00, 4.410e+00, 4.840e+00, 5.290e+00,
                5.760e+00, 6.250e+00, 6.760e+00, 7.290e+00, 7.840e+00, 8.410e+00,
                9.000e+00, 9.610e+00, 1.024e+01, 1.089e+01, 1.156e+01, 1.225e+01,
                1.296e+01, 1.369e+01, 1.444e+01, 1.521e+01, 1.600e+01, 1.681e+01,
                1.764e+01, 1.849e+01, 1.936e+01, 2.025e+01, 2.116e+01, 2.209e+01,
                2.

In [16]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=f(x),  name='f(x)=x<sup>2</sup>'))
fig.add_trace(go.Scatter(x=x, y=x, name='g(x)=x'))
fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  title="Plot Title",
                  hovermode="x",
                  xaxis_title="x Axis Title",
                  yaxis_title="y Axis Title",
                  margin=dict(l=0, r=80, t=40, b=0))
fig.update_traces(hoverinfo="all", hovertemplate="Аргумент: %{x}<br>Функция: %{y}")
fig.show()